In [210]:
import json
import os,requests
import pymongo
import pandas as pd
from sqlalchemy import create_engine,inspect, func
import psycopg2

In [82]:
term = 'Restaurants '
location = 'Toronto'
SEARCH_LIMIT = 50

In [178]:
url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format("tjoCWV1N7vncjx7yQkbRj8sa_V35ENjRSgetql-asev5rmOrqCjQEjVISjMN191l5KbOvmJfHz2u6Y-ch8agkzJ-46qeM8R2lMfrqZoLBTk02d4zHSi8QkhY9rY4XnYx"),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'offset':949
            }

In [179]:
response = requests.get(url, headers=headers, params=url_params)



In [180]:
print(response)
print(type(response.text))
print(response.text)

<Response [200]>
<class 'str'>
{"businesses": [{"id": "dp6Cb_S8uCzZK_h5tpq_Kg", "alias": "paracas-peruvian-toronto", "name": "Paracas Peruvian", "image_url": "https://s3-media3.fl.yelpcdn.com/bphoto/657voTDXMq989Id-ZiG5Ug/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/paracas-peruvian-toronto?adjust_creative=7XIiaRJ_izN-Q5jpSswUQA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=7XIiaRJ_izN-Q5jpSswUQA", "review_count": 22, "categories": [{"alias": "peruvian", "title": "Peruvian"}], "rating": 3.5, "coordinates": {"latitude": 43.6794411, "longitude": -79.436944}, "transactions": [], "price": "$$", "location": {"address1": "992 St.Clair Avenue W", "address2": null, "address3": "", "city": "Toronto", "zip_code": "M6E 1A2", "country": "CA", "state": "ON", "display_address": ["992 St.Clair Avenue W", "Toronto, ON M6E 1A2", "Canada"]}, "phone": "+14165325627", "display_phone": "+1 416-532-5627", "distance": 3277.41216038259}, {"id": "JOoblYsQjFT-47tkt6om0A",

In [181]:
for business in response.json()["businesses"]:
    name=business["name"]
    Resto_name.append(name)
    catg=business["categories"][0]["title"]
    Food_category.append(catg)
    rating=business["rating"]
    Resto_rating.append(rating)
    lat=business["coordinates"]["latitude"]
    Lat.append(lat)
    long=business["coordinates"]["longitude"]
    Long.append(long)
    addr=business["location"]["display_address"][0]
    Address.append(addr)
    city=business["location"]["city"]
    City.append(city)
    phn=business["display_phone"]
    Phone.append(phn)
   


In [196]:
len(Phone)

1000

In [183]:
 len(list(set(Resto_name)))

952

In [226]:
food_dict = {'rname': Resto_name, 
             'fcategory': Food_category,
             'Rating': Resto_rating,
             'lat': Lat,
             'long': Long,
             'address':Address,
             'city': City,
             'phone': Phone
             }

In [227]:
yelp_df = pd.DataFrame(food_dict)

In [230]:
connection_string = "postgres:Ruhi@localhost:5432/fooddb2"
engine = create_engine(f'postgresql+psycopg2://{connection_string}')

In [231]:
#engine.table_names()
inspector = inspect(engine)
inspector.get_table_names()

['yelpdata2']

In [232]:
yelp_df.to_sql(name='yelpdata1', con=engine, if_exists='append', index=True)

In [233]:
# read a table from database into pandas dataframe, replace "tablename" with your table name
df = pd.read_sql_table('yelpdata1',engine)

In [234]:
df

,index,rname,fcategory,Rating,lat,long,address,city,phone
0,0,Eataly,Italian,3.5,43.669615,-79.388795,55 Bloor Street W,Toronto,+1 437-374-0250
1,1,Steam Whistle Brewing,Breweries,4.5,43.641141,-79.385235,255 Bremner Boulevard,Toronto,+1 416-362-2337
2,2,Taro's Fish,Seafood Markets,4.5,43.769860,-79.374550,800 Sheppard Avenue E,Toronto,+1 416-730-8555
3,3,Northern Maverick Brewing Co,Brewpubs,3.5,43.645094,-79.402967,115 Bathurst Street,Toronto,+1 416-540-4030
4,4,Pai Northern Thai Kitchen,Thai,4.5,43.647840,-79.388720,18 Duncan Street,Toronto,+1 416-901-4724
...,...,...,...,...,...,...,...,...,...
995,995,Avocado Sushi,Japanese,4.0,43.667789,-79.388390,1105 Bay Street,Toronto,+1 416-323-9770
996,996,Bar Mercurio,Italian,3.0,43.667482,-79.400646,270 Bloor Street W,Toronto,+1 416-960-3877
997,997,Eggstatic,Breakfast & Brunch,4.5,43.705490,-79.375340,1568 bayview avenue,Toronto,+1 647-352-1568
998,998,Taste of China,Chinese,3.5,43.654000,-79.398910,338 Spadina Ave,Toronto,+1 416-348-8828


In [235]:
df.to_csv('data\yelp.csv')